# CQL Playground

## Overview

The **CQL Playground** allows users to interact with Maxar's imagery services through filtering and sorting capabilities via WFS and WMS. 
- With **WFS**, you can retrieve detailed metadata for basemap imagery that matches specific criteria, such as location, acquisition date, and cloud coverage.
- **WMS** provides a visual representation of the basemap imagery directly on a map. This tool enables efficient exploration and visualization of Maxar's Vivid Basemaps for research and analysis.

## API Key Validation Process

The following cell validates the Maxar API key provided by the user. It performs the following tasks:

1. **User Input for API Key**:  
   The user is prompted to securely input their Maxar API key, which is required to access Maxar services.

2. **Validate API Key**:  
   A specific URL is used to verify the API key's validity. While this endpoint is not explicitly designed for validation, it reliably indicates whether the key is valid:
   - If the server returns a `200` status code, the API key is valid.
   - Any other status code indicates the key is invalid.
     
3. **Send API Request**:  
   A `HEAD` request is sent to the URL, which efficiently checks the API key without retrieving unnecessary data.


In [3]:
import requests
from getpass import getpass

maxar_api_key = getpass("Enter your API Key")

url = "https://api.maxar.com/streaming/v1/ogc/ows?service=WFS&request=DescribeFeatureType&version=2.0.0&maxar_api_key="

headers = {
  'Accept': 'application/geo+json',
  'MAXAR-API-KEY': maxar_api_key
}

try:
    response = requests.head(url, headers=headers)
    
    if response.status_code == 200:
        print("API key is valid.")
    else:
        print(f"API key validation failed with status code: {response.status_code}")
        
except requests.RequestException as e:
    print(f"Error checking API key: {e}")

Enter your API Key ········


API key is valid.


## Key Parameters in the Search Panel

### Filters  
These parameters allow you to refine your search results based on specific criteria. Examples of how each filter is used are available in the search panel.

- **Catalog ID**: Display a specific image using its Catalog ID.  
  - Field: `legacyIdentifier`  
  - Example: `legacyIdentifier='1020010107DFF400'`

- **Date Range**: Show images within a specific date range.  
  - Field: `acquisitionDate`  
  - Example:  
    - `acquisitionDate > '2022-08-15' AND acquisitionDate < '2024-08-15'`

- **Feature ID**: Display images by their Feature ID, used for the image's vector footprint.  
  - Field: `featureId`  
  - Example: `featureId='99cdc53d-bba1-8eca-e19a-f25ddc4d540d'`

- **Ground Sample Distance**: Filter images by ground sample distance in meters (pixel resolution).  
  - Field: `groundSampleDistance`  
  - Example:  
    - `groundSampleDistance BETWEEN 0.7 AND 1.0`

- **Satellite Name**: Display images from specific satellites (e.g., WV01, WV02, GE01).  
  - Field: `source`  
  - Examples:  
    - `source = 'WV01'`  
    - `source IN ('WV03_VNIR', 'LG01')`  

- **Cloud Cover Percent**: Filter images by the percentage of cloud coverage.  
  - Field: `cloudCover`  
  - Example: `cloudCover < 20`

- **Off-nadir Angle**: Filter images by their off-nadir angle (lower values indicate straighter down views).  
  - Field: `offNadirAngle`  
  - Example: `offNadirAngle < 10`

- **Sun Angle**: Filter images by sun angle.  
  - Field: `sunElevation`  
  - Example: `sunElevation > 60`

---

### Sorts  
These parameters allow you to sort the search results. Examples of usage are provided in the search panel.

- **Date Range**: Sort results by acquisition date.  
  - Examples:  
    - Ascending: `acquisitionDate A`  
    - Descending: `acquisitionDate D`

- **Ground Sample Distance**: Sort results by ground sample distance.  
  - Examples:  
    - Ascending: `groundSampleDistance A`  
    - Descending: `groundSampleDistance D`


---

For more information, please refer to the [API Documentation](https://developers.maxar.com/docs/streaming-basemap/guides/basemap-api-guide).

## Define the `cql_filter`  and `sortBy` Parameters

The available options for filtering and sorting parameters are defined in the lists above. The values selected from those lists will be used in the filtering and sorting examples below.

### Filtering Parameters:
- Filtering parameters such as `acquisitionDate` and `cloudCover` are defined as variables. These values are stored in a list (`cql_filter_params`), and then joined using the `AND` operator to form the final `cql_filter` that will be used in the query.

### Sorting Parameters:
- Similarly, sorting parameters (e.g., `acquisitionDate`) are defined and stored in the list (`sort_by_params`). These are also joined using the AND operator to create the final sortBy.

In [4]:
#Filtering Parameters
acquisitionDate="BETWEEN '2022-08-15' AND '2024-08-15'"
cloudCover="< 20"

cql_filter_params = [
    f"acquisitionDate {acquisitionDate}",
    f"cloudCover {cloudCover}"
]

cql_filter = " AND ".join(cql_filter_params)


#Sorting Parameters
acquisitionDate_sort="D"

sort_by_params = [
    f"acquisitionDate {acquisitionDate_sort}"
]

sortBy = " AND ".join(sort_by_params)

print("CQL Filter:")
print(cql_filter)
print("\nSort By:")
print(sortBy)



CQL Filter:
acquisitionDate BETWEEN '2022-08-15' AND '2024-08-15' AND cloudCover < 20

Sort By:
acquisitionDate D


# Default Interactive Map

This is the **default map** displayed when starting the app. It is centered on Denver, Colorado. By adding **filter** and **sort parameters**, you will be able to interact with the map to visualize specific data later.


In [5]:
from ipyleaflet import Map, basemaps, basemap_to_tiles

default_map = Map(center=(39.57, -105.045), zoom=8)
tile_layer = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
default_map.add_layer(tile_layer)

default_map

Map(center=[39.57, -105.045], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

# Displaying Search Results on an Interactive Map

The following cell uses `ipyleaflet` to create an interactive map that allows you to explore imagery from the WMS API. The map dynamically loads and displays basemap imagery based on predefined filtering criteria when a button is clicked. Here's what the code does:

1. **Initialize the Map**:  
   Sets up a base map at a specific location with a zoom level of 8.

2. **Define the WMS Layer**:  
   Configures a `WMSLayer` to stream imagery from the Maxar API. The layer applies filtering criteria (e.g., `groundSampleDistance`, `cloudCover`, and `acquisitionDate`) using a `CQL` filter to ensure the results meet the specified conditions.

3. **Load Results**:  
   Clicking the button labeled "Load WMS →" triggers the map to load and display the WMS layer.


In [6]:
from ipyleaflet import Map, WMSLayer, basemaps, projections
from ipywidgets import Button, VBox
from IPython.display import display

m = Map(center=(39.57, -105.045), zoom=8)

wms_layer = WMSLayer(
    url=f"https://api.maxar.com/streaming/v1/ogc/wms?maxar_api_key={maxar_api_key}",
    layers="Maxar:Imagery",
    format="image/vnd.jpeg-png",
    transparent=True,
    version="1.3.0",
    crs=projections.EPSG4326,
    tile_size=512,
    cql_filter=cql_filter,
    sortBy=sortBy
)

# Button to load the WMS layer
load_button = Button(description="Load →")

def on_load_button_click(change):
    if wms_layer not in m.layers:
        m.add(wms_layer)
        print("WMS layer loaded.")
    else:
        print("WMS layer already loaded.")

load_button.on_click(on_load_button_click)

ui = VBox([m, load_button])
display(ui)


## Loading WFS Results

When a WMS request is made and the tiles load, each tile contains data. To access this data, simply click a tile on the map. This will trigger a call to the WFS endpoint to retrieve the information for the selected tile/coordinates. In the app, a popup displaying the data would appear on the screen. However, in this example, clicking on a tile will print the information below the map instead.


In [7]:
from ipyleaflet import Map, WMSLayer, projections
from ipywidgets import Output, VBox
import requests

wfs_url_template = (
    "https://api.maxar.com/streaming/v1/ogc/wfs?"
    "service=WFS&request=GetFeature&version=2.0.0&typeNames=Maxar:FinishedFeature"
    "&outputFormat=application/json"
    "&cql_filter=BBOX(featureGeometry,{bbox},'EPSG:4326') AND {cql_filter} "
    "&sortBy={sortBy}"
    "&maxar_api_key={maxar_api_key}"
)

m = Map(center=(39.57, -105.045), zoom=8)

wms_layer = WMSLayer(
    url=f"https://api.maxar.com/streaming/v1/ogc/wms?maxar_api_key={maxar_api_key}",
    layers="Maxar:Imagery",
    format="image/vnd.jpeg-png",
    transparent=True,
    version="1.3.0",
    crs=projections.EPSG4326,
    tile_size=512,
    cql_filter=cql_filter,
    sortBy=sortBy
)
m.add(wms_layer)

output = Output()

def handle_map_click(**kwargs):
    if kwargs.get("type") == "click":
        print("Clicking")
        lat, lng = kwargs["coordinates"]
        delta = 0.01  # Small delta for bbox calculation

        bbox = f"{lng-delta},{lat-delta},{lng+delta},{lat+delta}"

        # Construct WFS URL with dynamic bbox
        wfs_url = wfs_url_template.format(bbox=bbox, cql_filter=cql_filter, sortBy=sortBy, maxar_api_key=maxar_api_key)

        # Fetch data from WFS endpoint
        response = requests.get(wfs_url)
        with output:
            output.clear_output()
            if response.status_code == 200:
                try:
                    wfs_data = response.json()
                    features = wfs_data.get("features", [])
                    if features:
                        print("WFS Response:")
                        for feature in features:
                            feature_id = feature.get("id")
                            properties = feature.get("properties", {})
                            print(f"\nID: {feature_id}")
                            for key, value in properties.items():
                                print(f"  {key}: {value}")
                    else:
                        print("No data available for the selected location.")
                except requests.exceptions.JSONDecodeError:
                    print("Failed to parse WFS response.")
            else:
                print(f"Failed to retrieve WFS data. Status code: {response.status_code}")

# Attach click event handler to the map
m.on_interaction(handle_map_click)

ui = VBox([m, output])
ui
